In [7]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import pandas as pd
from sklearn import svm, model_selection, metrics, linear_model, tree
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.svm import SVC, SVR, LinearSVR
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectPercentile, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, LogisticRegression

from lightgbm import LGBMClassifier, LGBMRegressor
import sklearn.feature_selection as feature_selection
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras.layers import Embedding
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importing libraries for building the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras import regularizers      #for l2 regularization
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.utils import class_weight

In [10]:
acids = ["R", "H", "K", "D", "E", "S", "T", "N", "Q", "C", "U", "G","P", "A", "I", "L", "M", "F","W", "Y", "V"]
def seq_to_vec(seq):
    """amino acid sequence to a number (index in the array of acids)
       H - 1 index -> 1
    """
    vec = []
    for word in seq:
        vec.append(acids.index(word)) 
    return np.array(vec)

def hot_encoder(elem):
    """amino acid sequence to a hot vector of dim 21 (one in the place of index-number in the array of acids)
       H - 1 index -> 010000000000000000000
    """
    hot_vector = np.zeros(21)
    index = acids.index(elem)
    hot_vector[index] = 1
    return hot_vector

def hot_encoder_group(elem):
    """amino acid sequence to a hot vector of dim 21 (number of group in the place of index-number in the array of acids)
       D - 3 index and belongs to 2-d group; i -> 000200000000000000000;
    """
    hot_vector = np.zeros(21)
    index = acids.index(elem)
    group_index = group_amino(index + 1)
    hot_vector[index] = group_index + 1
    return hot_vector

def seq_to_hot(seq):
    vec = []
    for word in seq:
        vec.append(hot_encoder(word)) 
    return np.array(vec)

def seq_to_hot_group(seq):
    vec = []
    for word in seq:
        vec.append(hot_encoder_group(word)) 
    return np.array(vec)

    
def group_amino(idx):
    """return chemical groupping
       check this: https://commons.wikimedia.org/wiki/File:Amino_Acids-wide.svg
    """
    if (idx == 1 or idx == 2 or idx == 3 or idx == 4 or idx == 5):
        return 0
    elif (idx == 6 or idx == 7 or idx == 8 or idx == 9):
        return 1
    elif (idx == 10 or idx == 11 or idx == 12 or idx == 13):
        return 2
    elif (idx == 14 or idx == 15 or idx == 16 or idx == 17 or idx == 18 or idx == 19 or idx == 20 or idx == 21):
        return 3
    
def acids_to_channels(seq):
    """creates 4*4 feature tensor
       every group corresponds to a special channel. Firstly we encode protein into a seqience of indices
       of amino acids array.
       Eg: 2348
       In every "subdimension - channel" we leave just idxs of elements belonging to this group
       Eg 2 and 3 belongs to the first group, 4 to 2d and 8 to 4-th:
       2300
       0040
       0000
       0008
    """
    amino_seq = seq_to_vec(seq)
    tensor = np.zeros((4, 4))
    for group in range(4):
        for elem in range(4):
            if(group_amino(amino_seq[elem]) == group):
                tensor[group, elem] = amino_seq[elem]
    return tensor
    
def traint_test_split(X,y, size):
    train_length = int(size*X.shape[0])
    return X[:train_length], y[:train_length], X[train_length:], y[train_length:]

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def get_f1(y_true, y_pred): #taken from old keras source code
    #F1 metric for accuracy
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [22]:
X = pd.read_csv("data/train.csv")
X['hot_seq'] = X['Sequence'].transform(lambda x: seq_to_hot(x))
# X['hot_seq'] = X['Sequence'].transform(lambda x: seq_to_hot_group(x))
X_all = X['hot_seq'].to_numpy()
X_all = np.stack(X_all)
X_all = np.reshape(X_all, (X_all.shape[0], -1))
# X_all = X_all.reshape((X_all[0], X_all[1]* X_all[2])
y_all = X['Active'].to_numpy()

In [8]:

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_all),
                                                 y_all)

In [23]:
num_folds = 5
verbosity = 1
no_epochs = 100

# Cross validation with 5 folds and 3 dense layers (FC network)

In [24]:
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
for train, test in kfold.split(X_all, y_all):
    model = Sequential()
    model.add(Dense(500, input_dim = 84, activation='relu'))
#     model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
    model.add(Dense(500, activation='relu'))
#     model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
#     model.add(Dense(5000, input_dim = 500, kernel_initializer="normal", activation='relu'))
#     model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',get_f1]) 
#     model.summary()
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    history = model.fit(X_all[train], y_all[train],
              class_weight = class_weights,
              verbose=verbosity,
              epochs=no_epochs,
              validation_data=(X_all[test], y_all[test]))
    score = model.evaluate(X_all[test], y_all[test], verbose=0)
    print(f'F1 score for fold {fold_no}: {score[2]}')
    print('Test loss:', score[0])
#     print('Test accuracy:', score[1])
#     print('Test accuracy F1:', score[2])
    fold_no = fold_no + 1
    

------------------------------------------------------------------------
Training for fold 1 ...
Train on 89600 samples, validate on 22400 samples
Epoch 1/100
89600/89600 [==============================] - 11s 118us/step - loss: 0.0503 - accuracy: 0.9809 - get_f1: 0.4578 - val_loss: 0.0326 - val_accuracy: 0.9877 - val_get_f1: 0.5982
Epoch 2/100
89600/89600 [==============================] - 10s 109us/step - loss: 0.0263 - accuracy: 0.9894 - get_f1: 0.5861 - val_loss: 0.0284 - val_accuracy: 0.9894 - val_get_f1: 0.6012
Epoch 3/100
89600/89600 [==============================] - 10s 116us/step - loss: 0.0203 - accuracy: 0.9917 - get_f1: 0.6041 - val_loss: 0.0268 - val_accuracy: 0.9895 - val_get_f1: 0.6305
Epoch 4/100
89600/89600 [==============================] - 9s 99us/step - loss: 0.0152 - accuracy: 0.9939 - get_f1: 0.6369 - val_loss: 0.0264 - val_accuracy: 0.9912 - val_get_f1: 0.6388
Epoch 5/100
89600/89600 [==============================] - 8s 94us/step - loss: 0.0124 - accuracy: 0.99

Epoch 44/100
89600/89600 [==============================] - 8s 92us/step - loss: 2.0564e-04 - accuracy: 0.9999 - get_f1: 0.7081 - val_loss: 0.1284 - val_accuracy: 0.9912 - val_get_f1: 0.6476
Epoch 45/100
89600/89600 [==============================] - 8s 92us/step - loss: 6.4783e-04 - accuracy: 0.9998 - get_f1: 0.6938 - val_loss: 0.1254 - val_accuracy: 0.9914 - val_get_f1: 0.6343
Epoch 46/100
89600/89600 [==============================] - 8s 88us/step - loss: 3.5291e-04 - accuracy: 0.9998 - get_f1: 0.7000 - val_loss: 0.1245 - val_accuracy: 0.9920 - val_get_f1: 0.6390
Epoch 47/100
89600/89600 [==============================] - 8s 88us/step - loss: 5.1954e-04 - accuracy: 0.9999 - get_f1: 0.7024 - val_loss: 0.1162 - val_accuracy: 0.9923 - val_get_f1: 0.6508
Epoch 48/100
89600/89600 [==============================] - 8s 88us/step - loss: 3.4921e-04 - accuracy: 0.9999 - get_f1: 0.6881 - val_loss: 0.1223 - val_accuracy: 0.9923 - val_get_f1: 0.6485
Epoch 49/100
89600/89600 [===================

Epoch 87/100
89600/89600 [==============================] - 8s 92us/step - loss: 4.8404e-11 - accuracy: 1.0000 - get_f1: 0.7111 - val_loss: 0.1651 - val_accuracy: 0.9927 - val_get_f1: 0.6493
Epoch 88/100
89600/89600 [==============================] - 9s 98us/step - loss: 4.6580e-11 - accuracy: 1.0000 - get_f1: 0.6954 - val_loss: 0.1655 - val_accuracy: 0.9927 - val_get_f1: 0.6493
Epoch 89/100
89600/89600 [==============================] - 9s 101us/step - loss: 4.5630e-11 - accuracy: 1.0000 - get_f1: 0.7007 - val_loss: 0.1659 - val_accuracy: 0.9927 - val_get_f1: 0.6493
Epoch 90/100
89600/89600 [==============================] - 8s 92us/step - loss: 4.4536e-11 - accuracy: 1.0000 - get_f1: 0.7011 - val_loss: 0.1661 - val_accuracy: 0.9927 - val_get_f1: 0.6493
Epoch 91/100
89600/89600 [==============================] - 8s 91us/step - loss: 4.3227e-11 - accuracy: 1.0000 - get_f1: 0.7029 - val_loss: 0.1664 - val_accuracy: 0.9927 - val_get_f1: 0.6493
Epoch 92/100
89600/89600 [==================

Epoch 72/100
89600/89600 [==============================] - 8s 87us/step - loss: 8.0442e-11 - accuracy: 1.0000 - get_f1: 0.7204 - val_loss: 0.1242 - val_accuracy: 0.9917 - val_get_f1: 0.5924
Epoch 73/100
89600/89600 [==============================] - 8s 84us/step - loss: 7.7430e-11 - accuracy: 1.0000 - get_f1: 0.7021 - val_loss: 0.1244 - val_accuracy: 0.9917 - val_get_f1: 0.5924
Epoch 74/100
89600/89600 [==============================] - 8s 88us/step - loss: 7.6145e-11 - accuracy: 1.0000 - get_f1: 0.7150 - val_loss: 0.1247 - val_accuracy: 0.9917 - val_get_f1: 0.5924
Epoch 75/100
89600/89600 [==============================] - 9s 95us/step - loss: 7.4096e-11 - accuracy: 1.0000 - get_f1: 0.7029 - val_loss: 0.1250 - val_accuracy: 0.9918 - val_get_f1: 0.5929
Epoch 76/100
89600/89600 [==============================] - 8s 93us/step - loss: 7.3431e-11 - accuracy: 1.0000 - get_f1: 0.7075 - val_loss: 0.1252 - val_accuracy: 0.9917 - val_get_f1: 0.5924
Epoch 77/100
89600/89600 [===================

89600/89600 [==============================] - 8s 90us/step - loss: 0.0027 - accuracy: 0.9991 - get_f1: 0.6991 - val_loss: 0.0386 - val_accuracy: 0.9912 - val_get_f1: 0.5898
Epoch 15/100
89600/89600 [==============================] - 8s 88us/step - loss: 0.0018 - accuracy: 0.9994 - get_f1: 0.7117 - val_loss: 0.0466 - val_accuracy: 0.9917 - val_get_f1: 0.6134
Epoch 16/100
89600/89600 [==============================] - 8s 87us/step - loss: 0.0023 - accuracy: 0.9993 - get_f1: 0.6994 - val_loss: 0.0527 - val_accuracy: 0.9908 - val_get_f1: 0.6202
Epoch 17/100
89600/89600 [==============================] - 9s 95us/step - loss: 0.0018 - accuracy: 0.9994 - get_f1: 0.7133 - val_loss: 0.0468 - val_accuracy: 0.9917 - val_get_f1: 0.6139
Epoch 18/100
89600/89600 [==============================] - 8s 94us/step - loss: 0.0015 - accuracy: 0.9994 - get_f1: 0.6973 - val_loss: 0.0492 - val_accuracy: 0.9918 - val_get_f1: 0.6114
Epoch 19/100
89600/89600 [==============================] - 8s 87us/step - los

Epoch 99/100
89600/89600 [==============================] - 9s 102us/step - loss: 5.1700e-11 - accuracy: 1.0000 - get_f1: 0.7061 - val_loss: 0.1365 - val_accuracy: 0.9925 - val_get_f1: 0.6074
Epoch 100/100
89600/89600 [==============================] - 9s 98us/step - loss: 5.1941e-11 - accuracy: 1.0000 - get_f1: 0.7182 - val_loss: 0.1365 - val_accuracy: 0.9925 - val_get_f1: 0.6074
F1 score for fold 3: 0.607423722743988
Test loss: 0.13652959391403455
------------------------------------------------------------------------
Training for fold 4 ...
Train on 89600 samples, validate on 22400 samples
Epoch 1/100
89600/89600 [==============================] - 9s 105us/step - loss: 0.0502 - accuracy: 0.9810 - get_f1: 0.4546 - val_loss: 0.0333 - val_accuracy: 0.9862 - val_get_f1: 0.5669
Epoch 2/100
89600/89600 [==============================] - 8s 89us/step - loss: 0.0260 - accuracy: 0.9894 - get_f1: 0.5858 - val_loss: 0.0275 - val_accuracy: 0.9886 - val_get_f1: 0.6087
Epoch 3/100
89600/89600 [=

Epoch 84/100
89600/89600 [==============================] - 9s 97us/step - loss: 7.8811e-11 - accuracy: 1.0000 - get_f1: 0.7061 - val_loss: 0.1004 - val_accuracy: 0.9923 - val_get_f1: 0.6392
Epoch 85/100
89600/89600 [==============================] - 9s 101us/step - loss: 7.8638e-11 - accuracy: 1.0000 - get_f1: 0.6939 - val_loss: 0.1006 - val_accuracy: 0.9923 - val_get_f1: 0.6382
Epoch 86/100
89600/89600 [==============================] - 9s 101us/step - loss: 7.7915e-11 - accuracy: 1.0000 - get_f1: 0.6993 - val_loss: 0.1006 - val_accuracy: 0.9923 - val_get_f1: 0.6378
Epoch 87/100
89600/89600 [==============================] - 9s 96us/step - loss: 7.7920e-11 - accuracy: 1.0000 - get_f1: 0.7029 - val_loss: 0.1008 - val_accuracy: 0.9923 - val_get_f1: 0.6378
Epoch 88/100
89600/89600 [==============================] - 8s 95us/step - loss: 7.7391e-11 - accuracy: 1.0000 - get_f1: 0.6957 - val_loss: 0.1009 - val_accuracy: 0.9923 - val_get_f1: 0.6382
Epoch 89/100
89600/89600 [=================

Epoch 69/100
89600/89600 [==============================] - 8s 92us/step - loss: 9.0920e-11 - accuracy: 1.0000 - get_f1: 0.7118 - val_loss: 0.1057 - val_accuracy: 0.9922 - val_get_f1: 0.5970
Epoch 70/100
89600/89600 [==============================] - 9s 97us/step - loss: 9.0065e-11 - accuracy: 1.0000 - get_f1: 0.7229 - val_loss: 0.1058 - val_accuracy: 0.9922 - val_get_f1: 0.5970
Epoch 71/100
89600/89600 [==============================] - 8s 93us/step - loss: 8.8457e-11 - accuracy: 1.0000 - get_f1: 0.7193 - val_loss: 0.1060 - val_accuracy: 0.9922 - val_get_f1: 0.5970
Epoch 72/100
89600/89600 [==============================] - 8s 94us/step - loss: 8.7820e-11 - accuracy: 1.0000 - get_f1: 0.7236 - val_loss: 0.1061 - val_accuracy: 0.9922 - val_get_f1: 0.5970
Epoch 73/100
89600/89600 [==============================] - 8s 93us/step - loss: 8.7402e-11 - accuracy: 1.0000 - get_f1: 0.7064 - val_loss: 0.1062 - val_accuracy: 0.9922 - val_get_f1: 0.5970
Epoch 74/100
89600/89600 [===================

write answer

In [ ]:
x_final = pd.read_csv('data/test.csv')
x_final['Sequence'] = x_final['Sequence'].transform(lambda x: seq_to_hot(x))
x_final = np.stack(x_final['Sequence'].to_numpy())
x_final = np.reshape(x_final, (x_final.shape[0], -1))
answer = model.predict_classes(x_final)
np.savetxt("answer.csv", answer, delimiter=",", fmt='%d')

# Early stopping (doesn't help too much)

In [ ]:

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_all),
                                                 y_all)
num_folds = 5
verbosity = 1
no_epochs = 100

In [ ]:
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
for train, test in kfold.split(X_all, y_all):
    model = Sequential()
    model.add(Dense(1000, input_dim = 84, activation='relu'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
    model.add(Dense(1000, activation='relu'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
#     model.add(Dense(5000, input_dim = 500, kernel_initializer="normal", activation='relu'))
#     model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',get_f1]) 
#     model.summary()
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=0, mode='auto')]
    history = model.fit(X_all[train], y_all[train],
              class_weight = class_weights,
              verbose=verbosity,
              epochs=no_epochs,
              validation_data=(X_all[test], y_all[test]), callbacks = callbacks)
    score = model.evaluate(X_all[test], y_all[test], verbose=0)
    print(f'F1 score for fold {fold_no}: {score[2]}')
    print('Test loss:', score[0])
#     print('Test accuracy:', score[1])
#     print('Test accuracy F1:', score[2])
    fold_no = fold_no + 1
    